In [7]:
import pandas as pd

In [8]:
timetable = pd.read_csv("../data/final_micheal_timetable.csv",index_col=[0])
distribution_delays = pd.read_csv("../data/real_time_stats_arrival_period.csv",index_col=[0])
stops_15k = pd.read_csv("../data/stops_15k.csv",index_col=[0])

In [9]:
timetable[(timetable.dep_stop_id == "8503006:0:8") &(timetable.arr_stop_id == "8503340:0:1")].sort_values("dep_time",ascending=False)

,trip_id,dep_stop_id,arr_stop_id,dep_time,arr_time,mean,std,dayofweek,transport_name,trip_headsign,route_short_name
1327014,334.TA.26-7-A-j19-1.110.H,8503006:0:8,8503340:0:1,9:57,10:00,0,0,3,S-Bahn,Winterthur,7
2300645,334.TA.26-7-A-j19-1.110.H,8503006:0:8,8503340:0:1,9:57,10:00,0,0,5,S-Bahn,Winterthur,7
857286,334.TA.26-7-A-j19-1.110.H,8503006:0:8,8503340:0:1,9:57,10:00,0,0,2,S-Bahn,Winterthur,7
2812922,334.TA.26-7-A-j19-1.110.H,8503006:0:8,8503340:0:1,9:57,10:00,0,0,6,S-Bahn,Winterthur,7
3255222,334.TA.26-7-A-j19-1.110.H,8503006:0:8,8503340:0:1,9:57,10:00,0,0,7,S-Bahn,Winterthur,7
...,...,...,...,...,...,...,...,...,...,...,...
536934,336.TA.26-7-A-j19-1.110.H,8503006:0:8,8503340:0:1,10:27,10:30,0,0,1,S-Bahn,Winterthur,7
2969192,336.TA.26-7-A-j19-1.110.H,8503006:0:8,8503340:0:1,10:27,10:30,0,0,6,S-Bahn,Winterthur,7
3267677,337.TA.26-7-A-j19-1.113.H,8503006:0:8,8503340:0:1,10:27,10:30,0,0,7,S-Bahn,Winterthur,7
2472264,336.TA.26-7-A-j19-1.110.H,8503006:0:8,8503340:0:1,10:27,10:30,0,0,5,S-Bahn,Winterthur,7


In [10]:
timetable = timetable.reset_index()

In [11]:
timetable["hour_time"] = timetable.arr_time.apply(lambda x: int(x.split(":")[0]))

In [12]:
timetable = timetable[timetable["hour_time"] < 22]

In [13]:
timetable["time_period"] = timetable.hour_time.apply(lambda x: ((x + 2) // 4) - 2)

In [14]:
timetable.transport_name = timetable.transport_name.map({"Bus": "bus","RegioExpress":"zug","Tram": "tram","S-Bahn":"zug","Intercity": "zug",
                              "Standseilbahn":"zug","Schiff": "schiff","InterRegio": "zug","Taxi": "bus","Luftseilbahn":"luftseilbahn",
                             "Extrazug": "zug","Eurocity": "zug","Regionalzug":"zug"})

In [15]:
inter_df = timetable.merge(stops_15k,left_on="dep_stop_id",right_on="stop_id").drop(["latitude","longitude","stop_id"],axis=1).rename(columns={"stop_name": "dep_stop_name"})

In [16]:
inter_df = inter_df.merge(stops_15k,left_on="arr_stop_id",right_on="stop_id").drop(["latitude","longitude","stop_id"],axis=1).rename(columns={"stop_name": "arr_stop_name"})

In [17]:
final = inter_df.merge(distribution_delays,how="left",
                       left_on=["dep_stop_name","arr_stop_name","time_period","transport_name"],
                       right_on=["departure","arrival","arrival_period","transport_mode"])


In [18]:
final = final[["trip_id","dep_stop_id","arr_stop_id",
               "dep_stop_name","arr_stop_name","dep_time",
               "arr_time","delay_mean","delay_std",
               "dayofweek","time_period","transport_name",'trip_headsign','route_short_name']]

In [19]:
mean_all =  final.delay_mean.mean()
std_all = final.delay_std.mean()
final['delay_mean'] = final['delay_mean'].fillna(mean_all)
final['delay_std'] = final['delay_std'].fillna(std_all)

In [20]:
final = final.drop_duplicates()

In [21]:
final.dep_stop_id = final.dep_stop_id.apply(lambda x: x.split(":")[0])
final.arr_stop_id = final.arr_stop_id.apply(lambda x: x.split(":")[0])

In [ ]:
final = final.drop_duplicates(["dep_time","arr_time","dep_stop_name","arr_stop_name","transport_mode"])

In [22]:
final.to_csv("../data/full_timetable.csv")